## OGGM runs with differents climates and bias correction methods 

In [ ]:
# basic
import os 
import numpy as np
import xarray as xr
import geopandas as gpd

# oggm
from oggm import cfg, utils, workflow, graphics, tasks, shop
from oggm.shop import gcm_climate
from oggm.shop import millan22

# climate-related
import cftime
import regionmask
from xclim import sdba
from datetime import datetime

import warnings
warnings.simplefilter("ignore") 

chunks_dict = {"lon": 10, "lat": 10, "time": -1}

In [ ]:
# all basins in 9 selected zones 
basins = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_all.shp")[["Zone", "geometry"]]

# Glacier centroid location
ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6.shp")
ids = gpd.GeoDataFrame(ids, geometry=gpd.points_from_xy(ids.CenLon, ids.CenLat))

# Glaciers to run (filter by area)
ids = ids[ids.Area > 10]

# Select the glaciers inside (centroid) the basin (TODO: move to glacier outlet)
ids = gpd.clip(ids, basins, keep_geom_type = False)
ids = ids.sjoin(basins, how="inner", predicate='intersects')
ids_list = ids.RGIId
ids_zone = ids[["RGIId","Zone"]]

In [ ]:
cfg.initialize(logging_level='ERROR')

cfg.PARAMS['use_multiprocessing']  = True
cfg.PARAMS['baseline_climate']     = ''
cfg.PARAMS['prcp_scaling_factor']  = 1
cfg.PARAMS['hydro_month_sh']       = 1
cfg.PARAMS['hydro_month_nh']       = 1
cfg.PARAMS['border']               = 80
cfg.PARAMS['min_mu_star']          = 5
cfg.PARAMS['max_mu_star']          = 600
cfg.PARAMS['geodetic_mb_period']   = '2000-01-01_2020-01-01' 
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['continue_on_error']    = True
cfg.PARAMS['use_winter_prcp_factor']    = False

inversion_by_zone = True         # A factor by region or zone
file_id         = "PMET"         # Climate baseline
bias_correction = ["DQM", "MVA"] # Bias correction method
volume          = "Millan22_no"     # Reference volume dataset 

cfg.PATHS['working_dir']  = "/home/rooda/OGGM_results/" + file_id + "_run"
cfg.PATHS['climate_file'] = "/home/rooda/OGGM_results/"+ file_id +"_OGGM_1980_2019m.nc"

In [ ]:
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L1-L2_files/elev_bands/' # RGI 62
gdirs = workflow.init_glacier_directories(ids_list, from_prepro_level=2, prepro_border = 80, prepro_base_url = base_url)

In [ ]:
gdirs = workflow.init_glacier_directories(ids_list)

In [ ]:
# baseline climate tasks
baseline_period = slice("1980-01-01", "2014-12-31") # ISIMIP3b bias adjustment protocol

# prepare baseline climate for bias correction of projections 
pp_baseline  = xr.open_dataset(cfg.PATHS['climate_file'])["prcp"]
pp_baseline  = pp_baseline.sel(time=baseline_period)

t2m_baseline = xr.open_dataset(cfg.PATHS['climate_file'])["temp"]
t2m_baseline = t2m_baseline.sel(time=baseline_period)

# write climate file for each glacier
workflow.execute_entity_task(tasks.process_custom_climate_data, gdirs);

# calibration
utils.get_geodetic_mb_dataframe()
workflow.execute_entity_task(tasks.mu_star_calibration_from_geodetic_mb, gdirs); 
workflow.execute_entity_task(tasks.apparent_mb_from_any_mb, gdirs);

# inversion (for all the region or by zone)
if volume == "Millan22":
    workflow.execute_entity_task(millan22.thickness_to_gdir,gdirs);
    compile_volume = millan22.compile_millan_statistics(gdirs)
    compile_volume = compile_volume.replace([np.inf, -np.inf], np.nan, inplace=False)
    
    if inversion_by_zone:
        for zone in range(1,10):
            ids_subset = ids_zone[ids_zone.Zone == zone].RGIId.tolist()
            gdirs_subset = [gdir for gdir in gdirs if gdir.rgi_id in ids_subset]
            compile_volume_subset = compile_volume[compile_volume.index.isin(ids_subset)]
            workflow.calibrate_inversion_from_consensus(gdirs_subset, volume_m3_reference = compile_volume_subset.millan_vol_km3.sum()*1e9,
                                                        apply_fs_on_mismatch=True, error_on_mismatch=False, filter_inversion_output=True);
    else:
        workflow.calibrate_inversion_from_consensus(gdirs, volume_m3_reference = compile_volume.millan_vol_km3.sum()*1e9, 
                                                    apply_fs_on_mismatch=True, error_on_mismatch=False, filter_inversion_output=True);

else:
    if inversion_by_zone:
        for zone in range(1,10):
            gdirs_subset = [gdir for gdir in gdirs if gdir.rgi_id in ids_zone[ids_zone.Zone == zone].RGIId.tolist()]
            workflow.calibrate_inversion_from_consensus(gdirs_subset, apply_fs_on_mismatch=True, error_on_mismatch=False, filter_inversion_output=True);
    else:
        workflow.calibrate_inversion_from_consensus(gdirs, apply_fs_on_mismatch=True, error_on_mismatch=False, filter_inversion_output=True);

    
workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs); # ready to use
utils.compile_glacier_statistics(gdirs);  # compile few things

file_id = "_" + file_id
workflow.execute_entity_task(tasks.run_with_hydro, gdirs, 
                                run_task=tasks.run_from_climate_data,
                                store_monthly_hydro=True,
                                ys=2003,ye=2020,
                                output_filesuffix=file_id);

compile_run = utils.compile_run_output(gdirs, input_filesuffix=file_id)

start = datetime.now()

# future climate tasks
future_period = slice("2020-01-01", "2099-12-31") # period
#days = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])

os.chdir("/home/rooda/OGGM_results/Future_climate/")
gcm_list  = ["ACCESS-CM2", "EC-Earth3", "INM-CM5-0", "KACE-1-0-G", "MPI-ESM1-2-HR", 
             "CMCC-ESM2",  "GFDL-ESM4",  "IPSL-CM6A",  "MIROC6",   "NorESM2-LM"]
ssp_list  = ["ssp245","ssp370","ssp585"] 

gcm_list  = ["ACCESS-CM2"]
ssp_list  = ["ssp245"] 

for gcm in gcm_list:    
    for ssp in ssp_list:
        
        pp_model_ssp = xr.open_dataset("PP_" + gcm + "_" + ssp + ".nc")["pr"]
        pp_model_ssp = pp_model_ssp.interp(lat = pp_baseline.lat, lon = pp_baseline.lon)
        pp_model_ssp = pp_model_ssp.where(pp_baseline[0].notnull())
        pp_model_ssp = pp_model_ssp.chunk(chunks_dict)
              
        t2m_model_ssp = xr.open_mfdataset("T2M_" + gcm + "_" + ssp + ".nc")["tas"]
        t2m_model_ssp = t2m_model_ssp.interp(lat = t2m_baseline.lat, lon = t2m_baseline.lon)
        t2m_model_ssp = t2m_model_ssp.where(t2m_baseline[0].notnull())
        t2m_model_ssp = t2m_model_ssp.chunk(chunks_dict)
        
        for bc in bias_correction:
            
            if bc == "DQM":
                qdm_t2m = sdba.adjustment.QuantileDeltaMapping.train(ref = t2m_baseline, hist = t2m_model_ssp.sel(time = baseline_period), kind = "+", group="time.month")
                t2m_model_ssp_bc = qdm_t2m.adjust(t2m_model_ssp.sel(time = future_period), interp="nearest", extrapolation="constant")      

                qdm_pp  = sdba.adjustment.QuantileDeltaMapping.train(ref = pp_baseline, hist = pp_model_ssp.sel(time  = baseline_period), kind = "*", group="time.month")
                pp_model_ssp_bc  = qdm_pp.adjust(pp_model_ssp.sel(time  = future_period), interp="nearest", extrapolation="constant")
        
            if bc == "MVA":
                qdm_t2m = sdba.adjustment.Scaling.train(ref = t2m_baseline, hist = t2m_model_ssp.sel(time = baseline_period), kind = "+", group="time.month")
                t2m_model_ssp_bc = qdm_t2m.adjust(t2m_model_ssp.sel(time = future_period), interp="nearest")      

                qdm_pp  = sdba.adjustment.Scaling.train(ref = pp_baseline, hist = pp_model_ssp.sel(time  = baseline_period), kind = "*", group="time.month")
                pp_model_ssp_bc  = qdm_pp.adjust(pp_model_ssp.sel(time  = future_period), interp="nearest")
        
            t2m_model_ssp_bc = t2m_model_ssp_bc.rename("tas").transpose('time', 'lat', 'lon')         
            pp_model_ssp_bc  = pp_model_ssp_bc.rename("pr").transpose('time', 'lat', 'lon')
            t2m_model_ssp_bc.to_netcdf("/home/rooda/OGGM_results/T2M_Projection.nc")
            pp_model_ssp_bc.to_netcdf("/home/rooda/OGGM_results/PP_Projection.nc")
            rid = "_{}_{}_{}".format(gcm, ssp, bc)
        
            # write future climate file for each glacier
            workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs, filesuffix =  rid, 
                                     fpath_precip = "/home/rooda/OGGM_results/PP_Projection.nc", 
                                     fpath_temp = "/home/rooda/OGGM_results/T2M_Projection.nc", 
                                     apply_bias_correction=False);

            print(gcm, ssp, datetime.now()-start)
        
            # run the glacier using hydro function 
            workflow.execute_entity_task(tasks.run_with_hydro, gdirs, run_task = tasks.run_from_climate_data,
                                     climate_filename = 'gcm_data',  # use gcm_data, not climate_historical
                                     climate_input_filesuffix = rid,  # use the chosen scenario
                                     init_model_filesuffix = file_id,  # this is important! Start from 2020 glacier
                                     ref_geometry_filesuffix = file_id,  # also use this as area reference
                                     ref_area_from_y0 = True,  # and keep the same reference area as for the historical simulations
                                     output_filesuffix = rid,  # recognize the run for later
                                     store_monthly_hydro = True)  # add monthly diagnostics
            compile_run = utils.compile_run_output(gdirs, input_filesuffix=rid)

Process ForkPoolWorker-10:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/rooda/miniconda3/envs/oggm/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/rooda/miniconda3/envs/oggm/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/rooda/miniconda3/envs/oggm/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rooda/miniconda3/envs/oggm/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/rooda/miniconda3/envs/oggm/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rooda/miniconda3/envs/oggm/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/home/rooda/miniconda3/envs/oggm/lib/python3.9/multiprocessing/synchronize.py", line 95,